In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
q_votes = pd.DataFrame.from_csv('question-votes.csv', index_col=None)
q_votes.head(8)

,CreationVote,CreationPost,VoteType,PostId
0,2011-11-15T00:00:00.000,2011-11-15T20:45:51.037,2,1
1,2011-11-15T00:00:00.000,2011-11-15T20:45:51.037,2,1
2,2011-11-16T00:00:00.000,2011-11-15T20:45:51.037,2,1
3,2011-11-16T00:00:00.000,2011-11-15T20:45:51.037,2,1
4,2011-11-16T00:00:00.000,2011-11-15T20:45:51.037,2,1
5,2011-11-22T00:00:00.000,2011-11-15T20:45:51.037,2,1
6,2011-11-28T00:00:00.000,2011-11-15T20:45:51.037,2,1
7,2011-12-13T00:00:00.000,2011-11-15T20:45:51.037,2,1


In [3]:
q_votes.VoteType.value_counts()

2    10438
3      374
Name: VoteType, dtype: int64

In [13]:
q_votes['age'] = (pd.to_datetime(q_votes.CreationVote,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(q_votes.CreationPost,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
q_votes['age'] = q_votes['age'] + 1
#q_votes.age.value_counts()

In [15]:
q_votes_q = q_votes.sort_values(by='age').groupby(['PostId','age']).apply(lambda x: sum(x.VoteType==2) - sum(x.VoteType==3))

In [16]:
q_votes_q 

PostId  age 
1       0       2
        1       3
        7       1
        13      1
        28      1
        76      1
        157     1
        1401    1
        1405    1
2       0       6
        1       1
        4       1
        14      1
        76      1
        1317    1
        1444    1
3       0       6
        1       4
        2       1
        5       2
        6       1
        12      1
        62      1
        361     1
        614     1
        707     1
        775     1
        780     1
        781     1
        939     1
               ..
15625   0       1
        1       2
15627   1       3
15631   3       1
        4      -1
        9       1
15636   1       4
15639   0       1
        1       1
15644   2       2
15646   0       2
        1       1
15649   0       1
15651   1       2
        2       1
15655   1       1
        2       1
15657   0       2
        2       1
15662   0      -1
        1       1
15664   0       1
        2       1
15667   0      

In [17]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
mod_poi = smf.glm('ViewAge ~ Score + FavCount + AnsCount + CommentCount - 1 ', data=q_votes_q,
                  family=sm.families.Poisson(link=sm.families.links.log)).fit(method='bfgs')
#print mod_poi.mle_retvals['converged']
print mod_poi.summary()